In [1]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('action.h5')

# Actions to detect
actions = np.array(['halo', 'terima kasih', 'i love you'])

# Colors for visualization
colors = [(245, 117, 16), (117, 245, 16), (16, 117, 245)]

# Initialize MediaPipe Holistic model
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def draw_styled_landmarks(image, results):
    # Draw pose, face, and hands landmarks
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    if results.face_landmarks:
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp.solutions.face_mesh.FACEMESH_TESSELATION)
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33 * 4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468 * 3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21 * 3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21 * 3)
    return np.concatenate([pose, face, lh, rh])

def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0, 60 + num * 40), (int(prob * 100), 90 + num * 40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85 + num * 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    return output_frame

# Variables for detection
sequence = []
sentence = []
predictions = []
threshold = 0.5

# Open webcam
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Extract keypoints
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        # Prediction logic
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))

            # Visualization logic
            if np.unique(predictions[-10:])[0] == np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    if len(sentence) == 0 or actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5:
                sentence = sentence[-5:]

            # Display probabilities
            image = prob_viz(res, actions, image, colors)

        # Display the sentence
        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Show the frame
        cv2.imshow('OpenCV Feed', image)

        # Break condition
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 338ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
i love you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
i love you
1/1 ━━━━━━━━━━━━━━━